In [1]:
from torch import cuda, bfloat16
import torch
import transformers
from transformers import AutoTokenizer
from time import time
#import chromadb
#from chromadb.config import Settings
from langchain.llms import HuggingFacePipeline
from langchain.document_loaders import TextLoader,PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser

d:\Programs\Programming\BSUIR\NLIIS\LW6\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_id = 'model/Qwen1.5-4B-Chat/'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
# bnb_config = transformers.BitsAndBytesConfig(
#     load_in_8bit=True,  # Load model weights in 8-bit format
#     bnb_8bit_quant_type='nf8',  # Use nf8 quantization scheme for 8-bit
#     bnb_8bit_use_double_quant=True,  # Use double quantization for 8-bit
#     bnb_8bit_compute_dtype='bfloat16'  # Compute using bfloat16 for 8-bit quantization
# )

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

time_1 = time()
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
time_2 = time()
print(f"Prepare model, tokenizer: {round(time_2-time_1, 3)} sec.")

Loading checkpoint shards: 100%|██████████| 2/2 [00:09<00:00,  4.87s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Prepare model, tokenizer: 14.064 sec.


In [38]:
query_pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        torch_dtype=torch.float16,
        device_map="auto",
        # temperature=0.7,
        # max_new_tokens=200,
        # trust_remote_code=True,
)

The model 'Qwen2ForCausalLM' is not supported for question-answering. Supported models are ['AlbertForQuestionAnswering', 'BartForQuestionAnswering', 'BertForQuestionAnswering', 'BigBirdForQuestionAnswering', 'BigBirdPegasusForQuestionAnswering', 'BloomForQuestionAnswering', 'CamembertForQuestionAnswering', 'CanineForQuestionAnswering', 'ConvBertForQuestionAnswering', 'Data2VecTextForQuestionAnswering', 'DebertaForQuestionAnswering', 'DebertaV2ForQuestionAnswering', 'DistilBertForQuestionAnswering', 'ElectraForQuestionAnswering', 'ErnieForQuestionAnswering', 'ErnieMForQuestionAnswering', 'FalconForQuestionAnswering', 'FlaubertForQuestionAnsweringSimple', 'FNetForQuestionAnswering', 'FunnelForQuestionAnswering', 'GPT2ForQuestionAnswering', 'GPTNeoForQuestionAnswering', 'GPTNeoXForQuestionAnswering', 'GPTJForQuestionAnswering', 'IBertForQuestionAnswering', 'LayoutLMv2ForQuestionAnswering', 'LayoutLMv3ForQuestionAnswering', 'LEDForQuestionAnswering', 'LiltForQuestionAnswering', 'LlamaForQ

In [18]:
llm = HuggingFacePipeline(pipeline=query_pipeline)

In [36]:
from langchain.prompts import PromptTemplate
template = """Question: {question}\nAnswer: """
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

question = "What it is a human heart ?"

print(chain.invoke({"question": question}))

In [6]:
# loader = PyPDFLoader('text_corpus/Bergman_s_comprehensive_encyclopedia_of_human_anatomic_variation_Bergman_1_ed_2016.pdf')
# documents = loader.load()

# text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=20)
# all_splits = text_splitter.split_documents(documents)

In [7]:
# model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {"device": "cuda"}

# embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


In [8]:
# len(all_splits)

In [9]:
# vectordb = Chroma.from_documents(documents=all_splits[:100], embedding=embeddings, persist_directory="chroma_db")

In [10]:
torch.cuda.is_available()

True

In [11]:
# retriever = vectordb.as_retriever()

In [12]:
# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

In [13]:
# qa = RetrievalQA.from_chain_type(
#     llm=llm, 
#     chain_type="stuff", 
#     retriever=retriever, 
#     verbose=True
# )

In [14]:
# torch.cuda.empty_cache()

In [15]:
# result = qa.run('How can human anatomic variation can be defined? Keep it under 10 words')
# result